# This notebook shows how you can get upload a synonym map (created in a previous step) to an Azure Search service and associate it with a specific Azure Search index. This can be used to augment a downstream search operation. #

In [1]:
"""
See Azure Search REST API docs for more info:
    https://docs.microsoft.com/en-us/rest/api/searchservice/index
"""
import requests
import logging
import json
import pandas as pd

** Enter below your service and index names and API key **

In [2]:
# This is the service and index already created in Azure Portal
serviceName = "eyazuresearch2017"
indexName = "mh-eytaxidxer-synonymc512"
apiVersion = "2016-09-01-Preview"

# API key of the service subscription
apiKey = "XXXXXXXXXXXXXXXXXXXXXX"

** Enter below the synonym map file location **

In [3]:
INPUT_FILE = "keywords_synonym.txt"
file = open(INPUT_FILE,'r')
all_text = file.read()
file.close()

** Create a POST request to upload the synonym map to the existing search service above. Call it synonym-map. **

In [5]:
body= "{\"name\":\"synonym-map\",\"format\":\"solr\",\"synonyms\": \"%s\"}" %(all_text)

logging.basicConfig()
logging.getLogger().setLevel(logging.DEBUG)
requests_log = logging.getLogger("requests.packages.urllib3")
requests_log.setLevel(logging.DEBUG)
requests_log.propagate = True

def getServiceUrl():
    return "https://" + serviceName + ".search.windows.net"

def postMethod(body):
    headers = {"Content-type": "application/json", "api-key": apiKey}
    servicePath ="/synonymmaps?api-version=%s" %(apiVersion)
    r = requests.post(getServiceUrl() + servicePath, headers=headers, data=body)
    print(r, r.text)
    return r

r = postMethod(body)

DEBUG:requests.packages.urllib3.connectionpool:Starting new HTTPS connection (1): eyazuresearch2017.search.windows.net
DEBUG:requests.packages.urllib3.connectionpool:https://eyazuresearch2017.search.windows.net:443 "POST /synonymmaps?api-version=2016-09-01-Preview HTTP/1.1" 201 7415


<Response [201]> {"@odata.context":"https://eyazuresearch2017.search.windows.net/$metadata#synonymmaps/$entity","@odata.etag":"\"0x8D4BCC5C10D5AB5\"","name":"synonym-map","format":"solr","synonyms":"AAA, Accumulated Adjustment Account\nAcq., Acquiescence\nACRS, Accelerated Cost Recovery System\nADR, Asset Depreciation Range\nADLs, Activities of Daily Living\nADS, Alternative Depreciation System\nAFR, Applicable Federal Rate\nAGI, Adjusted Gross Income\nAIME, Average Indexed Monthly Earnings (Social Security)\nAMT, Alternative Minimum Tax\nAOD, Action on Decision\nARM, Adjustable Rate Mortgage\nATG, Audit Techniques Guide\nCB, Cumulative Bulletin\nCCA, Chief Council Advice\nCC-ITA, Chief Council - Income Tax and Accounting\nCCC, Commodity Credit Corporation\nCCP, Counter-Cyclical Program (government farm program)\nCDHP, Consumer-Driven Health Plan\nCFR, Code of Federal Regulations\nCLT, Charitable Lead Trust\nCOBRA, Consolidated Omnibus Budget Reconciliations Act of 1985\nCOGS, Cost of 

** Create a PUT request to associate the uploaded synonym map with a specific index defined at the top. **

In [6]:
put_body =json.dumps({
        "name": indexName,  
        "fields": [
        {"name": "Index", "type": "Edm.String", "key": True, "retrievable": True, "searchable": False, "filterable": False, "sortable": True, "facetable": False},

        {"name": "File", "type": "Edm.String", "retrievable": True, "searchable": False, "filterable": True, "sortable": True, "facetable": False},

        {"name": "Chapter", "type": "Edm.String", "retrievable": True, "searchable": False, "filterable": True, "sortable": True, "facetable": False},

        {"name": "Title", "type": "Edm.String", "retrievable": True, "searchable": True, "filterable": True, "sortable": True, "facetable": True},

        {"name": "SectionTitle", "type": "Edm.String", "retrievable": True, "searchable": True, "filterable": True, "sortable": False, "facetable": True},

        {"name": "SubsectionTitle", "type": "Edm.String", "retrievable": True, "searchable": True, "filterable": True, "sortable": True, "facetable": False},

        {"name": "Source", "type": "Edm.String", "retrievable": True, "searchable": False, "filterable": False, "sortable": True, "facetable": True},

        {"name": "FeatureType", "type": "Edm.String", "retrievable": True, "searchable": False, "filterable": True, "sortable": True, "facetable": True},

        {"name": "ParaText", "type": "Edm.String", "retrievable": True, "searchable": True, "filterable": False, "sortable": False, "facetable": False, "analyzer": "en.microsoft"},

        {"name": "Keywords", "type": "Edm.String", "retrievable": True, "searchable": True, "filterable": False, "sortable": False, "facetable": False, "analyzer": "en.microsoft","synonymMaps":["synonym-map"]}
        ]
    })

In [8]:
def putMethod(body):
    headers = {"Content-type": "application/json", "api-key": apiKey}
    servicePath ="/indexes/%s?api-version=%s" %(indexName,apiVersion)
    r = requests.put(getServiceUrl() + servicePath, headers=headers, data=put_body)
    print(r, r.text)
    return r

r = putMethod(body)

DEBUG:requests.packages.urllib3.connectionpool:Starting new HTTPS connection (1): eyazuresearch2017.search.windows.net
DEBUG:requests.packages.urllib3.connectionpool:https://eyazuresearch2017.search.windows.net:443 "PUT /indexes/mh-eytaxidxer-synonymc512?api-version=2016-09-01-Preview HTTP/1.1" 204 0


<Response [204]> 


** Check if the synonym map was uploaded successfully. **

In [10]:
def getSynonyms():
    #servicePath = '/indexers/%s?api-version=%s' % (indexName, apiVersion)
    headers = {"Content-type": "application/json", "api-key": apiKey}
    r = requests.get("https://eyazuresearch2017.search.windows.net/synonymmaps/synonym-map?api-version=2016-09-01-Preview", headers=headers)
    print(r, r.text)
    return r

getSynonyms()

DEBUG:requests.packages.urllib3.connectionpool:Starting new HTTPS connection (1): eyazuresearch2017.search.windows.net
DEBUG:requests.packages.urllib3.connectionpool:https://eyazuresearch2017.search.windows.net:443 "GET /synonymmaps/synonym-map?api-version=2016-09-01-Preview HTTP/1.1" 200 4230


<Response [200]> {"@odata.context":"https://eyazuresearch2017.search.windows.net/$metadata#synonymmaps/$entity","@odata.etag":"\"0x8D4BCC5C10D5AB5\"","name":"synonym-map","format":"solr","synonyms":"AAA, Accumulated Adjustment Account\nAcq., Acquiescence\nACRS, Accelerated Cost Recovery System\nADR, Asset Depreciation Range\nADLs, Activities of Daily Living\nADS, Alternative Depreciation System\nAFR, Applicable Federal Rate\nAGI, Adjusted Gross Income\nAIME, Average Indexed Monthly Earnings (Social Security)\nAMT, Alternative Minimum Tax\nAOD, Action on Decision\nARM, Adjustable Rate Mortgage\nATG, Audit Techniques Guide\nCB, Cumulative Bulletin\nCCA, Chief Council Advice\nCC-ITA, Chief Council - Income Tax and Accounting\nCCC, Commodity Credit Corporation\nCCP, Counter-Cyclical Program (government farm program)\nCDHP, Consumer-Driven Health Plan\nCFR, Code of Federal Regulations\nCLT, Charitable Lead Trust\nCOBRA, Consolidated Omnibus Budget Reconciliations Act of 1985\nCOGS, Cost of 

<Response [200]>